In [1]:
import os
import math
import datetime

from tqdm import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'2.0.0'

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
if tf.__version__.startswith("1."):
    tf.enable_eager_execution()


In addition to the standard libraries we imported above, we'll need to install the [bert-for-tf2](https://github.com/kpe/bert-for-tf2) python package, and do the imports required for loading the pre-trained weights and tokenizing the input text. 

In [5]:
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

In [6]:
bert_ckpt_dir="data/uncased_L-12_H-768_A-12/"
bert_ckpt_file = bert_ckpt_dir + "bert_model.ckpt"
bert_config_file = bert_ckpt_dir + "bert_config.json"

In [7]:
path = '/media/moh/abunayla/Disaster_Tweets/Notebooks/data/'


train = pd.read_csv(path + 'train.csv', usecols = ['text', 'target'])

submission_test = pd.read_csv(path + 'test.csv', usecols = ['text'])

sumission_df = pd.read_csv(path +'sample_submission.csv', index_col = 'id')

In [8]:
def load_datasets():
    dfs = np.array_split(train, 2)
    return dfs[0], dfs[1]

In [9]:
class DisasterTweets:
    DATA_COLUMN = "text"
    LABEL_COLUMN = "target"
    
    
    def __init__(self, tokenizer: FullTokenizer, sample_size=None, max_seq_len=1024):
        
        self.tokenizer = tokenizer
        
        self.sample_size = sample_size
        
        self.max_seq_len = 0
        
        train, test = load_datasets()
        
        train, test = map(lambda df: df.reindex(df[DisasterTweets.DATA_COLUMN].str.len().sort_values().index), [train, test])
        
        if sample_size is not None:
            assert sample_size % 128 == 0
            train, test = train.head(sample_size), test.head(sample_size)
            
            
        ((self.train_x, self.train_y),(self.test_x, self.test_y)) = map(self._prepare, [train, test])
        
        print("max seq_len", self.max_seq_len)
        
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        
        ((self.train_x, self.train_x_token_types),
         (self.test_x, self.test_x_token_types)) = map(self._pad, 
                                                       [self.train_x, self.test_x])
        
    def _prepare(self, df):
        x, y = [], []
        with tqdm(total=df.shape[0], unit_scale=True) as pbar:
            for ndx, row in df.iterrows():
                text, label = row[DisasterTweets.DATA_COLUMN], row[DisasterTweets.LABEL_COLUMN]
                tokens = self.tokenizer.tokenize(text)
                tokens = ["[CLS]"] + tokens + ["[SEP]"]
                token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
                self.max_seq_len = max(self.max_seq_len, len(token_ids))
                x.append(token_ids)
                y.append(int(label))
                pbar.update()
        return np.array(x), np.array(y)
    
    
    
    def _pad(self, ids):
        x, t = [], []
        token_type_ids = [0] * self.max_seq_len
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
            t.append(token_type_ids)
        return np.array(x), np.array(t)

In [10]:
%%time

tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))
data = DisasterTweets(tokenizer, 
                       sample_size=10*128*2,#5000, 
                       max_seq_len=128)

100%|██████████| 2.56k/2.56k [00:00<00:00, 3.21kit/s]
100%|██████████| 2.56k/2.56k [00:00<00:00, 2.94kit/s]


max seq_len 70
CPU times: user 1.77 s, sys: 32.7 ms, total: 1.8 s
Wall time: 1.8 s


In [11]:
def flatten_layers(root_layer):
    if isinstance(root_layer, keras.layers.Layer):
        yield root_layer
    for layer in root_layer._layers:
        for sub_layer in flatten_layers(layer):
            yield sub_layer


def freeze_bert_layers(l_bert):
    """
    Freezes all but LayerNorm and adapter layers - see arXiv:1902.00751.
    """
    for layer in flatten_layers(l_bert):
        if layer.name in ["LayerNorm", "adapter-down", "adapter-up"]:
            layer.trainable = True
        elif len(layer._layers) == 0:
            layer.trainable = False
        l_bert.embeddings_layer.trainable = False


def create_learning_rate_scheduler(max_learn_rate=5e-5,
                                   end_learn_rate=1e-7,
                                   warmup_epoch_count=10,
                                   total_epoch_count=90):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = (max_learn_rate/warmup_epoch_count) * (epoch + 1)
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler

In [12]:
def create_model(max_seq_len, adapter_size=64):
    """Creates a classification model."""
    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = adapter_size
        bert = BertModelLayer.from_params(bert_params, name="bert")
        
    input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="input_ids")
    
    output         = bert(input_ids)
    
    
    print("bert shape", output.shape)
    
    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(output)
    
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    
    logits = keras.layers.Dropout(0.5)(logits)
    
    logits = keras.layers.Dense(units=2, activation="softmax")(logits)

    model = keras.Model(inputs=input_ids, outputs=logits)
    
    model.build(input_shape=(None, max_seq_len))

    # load the pre-trained model weights
    load_stock_weights(bert, bert_ckpt_file)
    
    # freeze weights if adapter-BERT is used
    
    if adapter_size is not None:
        freeze_bert_layers(bert)
        
        model.compile(optimizer=keras.optimizers.Adam(),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])
        
        
    model.compile(optimizer=keras.optimizers.Adam(),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

    model.summary()
    return model

In [13]:

adapter_size = None # use None to fine-tune all of BERT
model = create_model(data.max_seq_len, adapter_size=adapter_size)

bert shape (None, 70, 768)
Done loading 196 BERT weights from: data/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7fcb63fe7b10> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 70)]              0         
_________________________________________________________________
bert (BertModelLayer)        (None, 70, 768)           1

In [ ]:
%%time

log_dir = ".log/disaster_tweets/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

total_epoch_count = 5
# model.fit(x=(data.train_x, data.train_x_token_types), y=data.train_y,
model.fit(x=data.train_x, y=data.train_y,
          validation_split=0.1,
          batch_size=48,
          shuffle=True,
          epochs=total_epoch_count,
          callbacks=[create_learning_rate_scheduler(max_learn_rate=1e-5,
                                                    end_learn_rate=1e-7,
                                                    warmup_epoch_count=20,
                                                    total_epoch_count=total_epoch_count),
                     keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True),
                     tensorboard_callback])

Train on 2304 samples, validate on 256 samples

Epoch 00001: LearningRateScheduler reducing learning rate to 5.000000000000001e-07.
Epoch 1/5
2304/2304 [==============================] - 561s 244ms/sample - loss: 0.7299 - acc: 0.5035 - val_loss: 0.6818 - val_acc: 0.5938

Epoch 00002: LearningRateScheduler reducing learning rate to 1.0000000000000002e-06.
Epoch 2/5
2304/2304 [==============================] - 553s 240ms/sample - loss: 0.6790 - acc: 0.5846 - val_loss: 0.7042 - val_acc: 0.5703

Epoch 00003: LearningRateScheduler reducing learning rate to 1.5000000000000002e-06.
Epoch 3/5
2304/2304 [==============================] - 554s 240ms/sample - loss: 0.6506 - acc: 0.6293 - val_loss: 0.6691 - val_acc: 0.5781

Epoch 00004: LearningRateScheduler reducing learning rate to 2.0000000000000003e-06.
Epoch 4/5
1536/2304 [===================>..........] - ETA: 2:58 - loss: 0.6320 - acc: 0.6380